In [1]:
from model import GAT
from train import train,test
import argparse
import torch
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T

In [2]:
def main(dataset_name,hidden_chanels,num_layers,dropout):
    epochs = 500
    hidden_chanels = int(hidden_chanels)
    num_layers = int(num_layers)
    epochs = int(epochs)
    torch.manual_seed(2020)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = PygNodePropPredDataset(name=dataset_name,transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    data = data.to(device)
    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)
    model = GAT(data.num_features, hidden_chanels,dataset.num_classes, num_layers,dropout).to(device)
    evaluator = Evaluator(name=dataset_name)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    valid_acc_list = []
    for epoch in range(1, epochs):
                loss = train(model, data, train_idx, optimizer)
                result = test(model, data, split_idx, evaluator)
                train_acc, valid_acc,test_acc = result
                valid_acc_list.append(valid_acc)
                if epoch % 10 == 0:
                    train_acc, valid_acc,test_acc = result
                    print(f'Epoch: {epoch:02d}, '
                          f'Loss: {loss:.4f}, '
                          f'Train: {100 * train_acc:.2f}%, '
                          f'Valid: {100 * valid_acc:.2f}% '
                          f'Test: {100 * test_acc:.2f}% ')
    return max(valid_acc_list)

In [3]:
from functools import partial

database = 'ogbn-arxiv'
fit_with_partial = partial(main,database)

In [4]:
pbounds = {'hidden_chanels':(10,100),'num_layers':(2,7),'dropout':(0.2,0.5)}

In [5]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [ ]:
optimizer.maximize(init_points=10, n_iter=30,)
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)
f = open("params.txt", "a")
f.write(optimizer.max)
f.close()